<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%8B%9C%EC%8A%A4%ED%85%9C_V3_(OAC_%EA%B0%9C%EB%B3%84_%EB%AA%A8%EB%8D%B8%2C_%EA%B0%80%EC%8A%B5_%EC%98%A8%EB%8F%84_SP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# import joblib # 실제 모델 저장/로드를 위해 사용 가능

class EnthalpyCalculator:
    def calculate_enthalpy(self, temperature_c, relative_humidity_percent, pressure_pa=101325):
        # 매우 단순화된 placeholder 유지
        return (temperature_c * 1000 + (relative_humidity_percent / 100) * 2500 * 10) * (1 + (temperature_c / 100))

    def calculate_temp_humidity_from_enthalpy(self, enthalpy_j_kg, initial_temp_c, pressure_pa=101325):
        initial_h_approx = self.calculate_enthalpy(initial_temp_c, 50)
        delta_h = enthalpy_j_kg - initial_h_approx
        cp_approx = 1025
        delta_temp_c = delta_h / cp_approx
        new_temp_c = initial_temp_c + delta_temp_c
        new_humidity_percent = 50 + (enthalpy_j_kg - initial_h_approx) / 20000.0 * 5
        new_humidity_percent = np.clip(new_humidity_percent, 5.0, 99.0)
        new_temp_c = np.clip(new_temp_c, -15.0, 55.0)
        return new_temp_c, new_humidity_percent

# --- 상세 더미 데이터 생성 함수 ---
def generate_dummy_data(num_rows=1000, num_oac=14, num_low_ch=8, num_high_ch=8, num_ct=9):
    """학습 및 테스트를 위한 상세 더미 시계열 데이터를 생성합니다."""
    print(f"상세 더미 데이터 생성 시작: {num_rows} 행, OAC {num_oac}대, 저온냉동기 {num_low_ch}대, 고온냉동기 {num_high_ch}대, 냉각탑 {num_ct}대")
    data_dict = {}

    datetimes = pd.to_datetime('2023-01-01') + pd.to_timedelta(np.arange(num_rows) * 1, unit='h')
    data_dict['datetime'] = datetimes

    outdoor_temps = 15 + 10 * np.sin(2 * np.pi * np.arange(num_rows) / (24 * 30)) + 5 * np.sin(2 * np.pi * np.arange(num_rows) / 24) + np.random.normal(0, 1, num_rows)
    outdoor_rhs = 60 + 15 * np.sin(2 * np.pi * np.arange(num_rows) / (24*15)) + 10 * np.sin(2*np.pi*np.arange(num_rows)/24 + np.pi/2) + np.random.normal(0,5,num_rows)
    outdoor_rhs = np.clip(outdoor_rhs, 20, 95)
    data_dict['outdoor_temp_c'] = outdoor_temps
    data_dict['outdoor_rh_percent'] = outdoor_rhs

    enthalpy_calc = EnthalpyCalculator()
    oac_coil_types = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']
    design_delta_h_oac = {'preheating': 15000, 'precooling': -20000, 'cooling': -25000, 'reheating': 12000, 'humidification': 8000} # 가습 시 엔탈피 증가 (주로 잠열 + 약간의 현열)

    total_precooling_load_watts_series = np.zeros(num_rows)
    total_cooling_load_watts_series = np.zeros(num_rows)

    for i in range(num_oac): # 각 OAC 유닛에 대해
        for row_idx in range(num_rows):
            oac_in_temp = outdoor_temps[row_idx] # 각 OAC는 동일 외기에서 시작
            oac_in_rh = outdoor_rhs[row_idx]

            for coil_name in oac_coil_types:
                sp_temp_key = f'oac_{i}_{coil_name}_set_point_temp_c' # 가습 포함 모든 코일 목표 온도
                open_rate_key = f'oac_{i}_{coil_name}_coil_open_rate'
                outlet_temp_key = f'oac_{i}_{coil_name}_후단온도_c'
                outlet_rh_key = f'oac_{i}_{coil_name}_후단습도_percent'

                if row_idx == 0: # 컬럼 초기화
                    data_dict[sp_temp_key] = np.zeros(num_rows)
                    data_dict[open_rate_key] = np.zeros(num_rows)
                    data_dict[outlet_temp_key] = np.zeros(num_rows)
                    data_dict[outlet_rh_key] = np.zeros(num_rows)

                # Set point 설정 (온도 기반)
                set_point_temp_val = 0
                if coil_name == 'preheating': set_point_temp_val = np.where(oac_in_temp < 18, 20, oac_in_temp) + np.random.normal(0,0.2)
                elif coil_name == 'precooling': set_point_temp_val = np.where(oac_in_temp > 25, 22, oac_in_temp) + np.random.normal(0,0.2)
                elif coil_name == 'cooling': set_point_temp_val = 12.0 + np.random.normal(0,0.2)
                elif coil_name == 'reheating': set_point_temp_val = 22.0 + np.random.normal(0,0.2)
                elif coil_name == 'humidification':
                    # 가습 목표 온도: 현재 온도보다 약간 높게, 하지만 과도하지 않게. 겨울철에만 주로 작동.
                    if outdoor_temps[row_idx] < 15 : # 겨울철 가정
                        set_point_temp_val = oac_in_temp + np.random.uniform(0.5, 2.0) # 현재보다 0.5~2도 높게
                    else: # 그 외 계절엔 거의 현재 온도 유지 (가습 안함)
                        set_point_temp_val = oac_in_temp

                data_dict[sp_temp_key][row_idx] = set_point_temp_val

                open_rate = 0.0
                inlet_h = enthalpy_calc.calculate_enthalpy(oac_in_temp, oac_in_rh)

                # 목표 온도 도달을 위한 필요 엔탈피 변화량 계산 (가습 포함 모든 코일)
                # 가습 코일의 경우, 목표 온도에 도달하기 위해 주로 습도를 높여 엔탈피를 증가시킨다고 가정
                target_rh_for_sp_temp = oac_in_rh # 단순화를 위해 목표 온도 도달 시 습도는 현재와 유사하다고 가정 (실제론 변함)
                if coil_name == 'humidification' and set_point_temp_val > oac_in_temp: # 가습으로 온도 상승 시
                    target_rh_for_sp_temp = min(oac_in_rh + (set_point_temp_val - oac_in_temp) * 5, 95) # 온도 1도당 RH 5% 증가 가정 (매우 단순화)

                target_h = enthalpy_calc.calculate_enthalpy(set_point_temp_val, target_rh_for_sp_temp)
                required_delta_h = target_h - inlet_h

                if design_delta_h_oac[coil_name] != 0:
                    if (design_delta_h_oac[coil_name] > 0 and required_delta_h > 0) or \
                       (design_delta_h_oac[coil_name] < 0 and required_delta_h < 0): # 코일 방향과 필요 방향 일치
                        open_rate = abs(required_delta_h / design_delta_h_oac[coil_name])

                open_rate = np.clip(open_rate, 0, 1) * np.random.uniform(0.9, 1.1)
                open_rate = np.clip(open_rate, 0, 1)
                data_dict[open_rate_key][row_idx] = open_rate

                actual_delta_h = design_delta_h_oac[coil_name] * open_rate
                outlet_h = inlet_h + actual_delta_h
                oac_out_temp, oac_out_rh = enthalpy_calc.calculate_temp_humidity_from_enthalpy(outlet_h, oac_in_temp)

                data_dict[outlet_temp_key][row_idx] = oac_out_temp
                data_dict[outlet_rh_key][row_idx] = oac_out_rh

                oac_in_temp = oac_out_temp # 다음 코일의 입구
                oac_in_rh = oac_out_rh

                air_mass_flow_per_oac = 12.0
                if coil_name == 'precooling':
                    total_precooling_load_watts_series[row_idx] += abs(actual_delta_h * air_mass_flow_per_oac)
                elif coil_name == 'cooling':
                    total_cooling_load_watts_series[row_idx] += abs(actual_delta_h * air_mass_flow_per_oac)

            if row_idx == 0:
                data_dict[f'oac_{i}_토출온도_c'] = np.zeros(num_rows)
                data_dict[f'oac_{i}_토출노점온도_c'] = np.zeros(num_rows)
                data_dict[f'oac_{i}_토출압력_pa'] = np.zeros(num_rows)

            data_dict[f'oac_{i}_토출온도_c'][row_idx] = oac_out_temp
            data_dict[f'oac_{i}_토출노점온도_c'][row_idx] = oac_out_temp - (100 - oac_out_rh)/5 # 매우 단순한 노점 근사
            data_dict[f'oac_{i}_토출압력_pa'][row_idx] = 101325 + np.random.uniform(100,300)

    data_dict['total_precooling_load_watts'] = total_precooling_load_watts_series
    data_dict['total_cooling_load_watts'] = total_cooling_load_watts_series

    for temp_type_prefix, num_chillers in [("low_temp_chiller", num_low_ch), ("high_temp_chiller", num_high_ch)]:
        load_key = 'total_cooling_load_watts' if "low_temp" in temp_type_prefix else 'total_precooling_load_watts'

        data_dict[f'{temp_type_prefix}_main_차압압력_bar'] = 1.5 + np.random.normal(0, 0.1, num_rows) - (data_dict[load_key]/(np.mean(data_dict[load_key]) if np.mean(data_dict[load_key])!=0 else 1) -1)*0.2
        data_dict[f'{temp_type_prefix}_main_차압개도율_percent'] = 60 + np.random.normal(0, 5, num_rows) + (data_dict[load_key]/(np.mean(data_dict[load_key]) if np.mean(data_dict[load_key])!=0 else 1) -1)*10
        data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'] = 50 + np.random.normal(0, 5, num_rows) + (data_dict[load_key]/(np.mean(data_dict[load_key]) if np.mean(data_dict[load_key])!=0 else 1) -1)*8
        data_dict[f'{temp_type_prefix}_main_supply_압력_pa'] = 200000 + np.random.uniform(-5000, 5000, num_rows)
        data_dict[f'{temp_type_prefix}_main_supply_온도_c'] = (7.0 if "low_temp" in temp_type_prefix else 10.0) + np.random.normal(0, 0.2, num_rows)
        data_dict[f'{temp_type_prefix}_main_return_압력_pa'] = 150000 + np.random.uniform(-5000, 5000, num_rows)
        data_dict[f'{temp_type_prefix}_main_return_온도_c'] = data_dict[f'{temp_type_prefix}_main_supply_온도_c'] + (data_dict[load_key] / (num_chillers * (200000 if "low_temp" in temp_type_prefix else 250000) + 1e-6)) * 5

        data_dict[f'{temp_type_prefix}_main_차압압력_bar'] = np.clip(data_dict[f'{temp_type_prefix}_main_차압압력_bar'], 1.0, 2.5)
        data_dict[f'{temp_type_prefix}_main_차압개도율_percent'] = np.clip(data_dict[f'{temp_type_prefix}_main_차압개도율_percent'], 10, 100)
        data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'] = np.clip(data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'], 10, 100)
        data_dict[f'{temp_type_prefix}_main_return_온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_main_return_온도_c'],
                                                                data_dict[f'{temp_type_prefix}_main_supply_온도_c']+0.5 ,
                                                                data_dict[f'{temp_type_prefix}_main_supply_온도_c']+10)

        active_count_series = np.ceil(data_dict[load_key] / (200000 if "low_temp" in temp_type_prefix else 250000)).astype(int)
        active_count_series = np.clip(active_count_series, 1, num_chillers)
        data_dict[f'{temp_type_prefix}_active_count'] = active_count_series

        for j in range(num_chillers):
            is_active_prob = active_count_series / num_chillers
            is_active_roll = np.random.rand(num_rows) < is_active_prob

            base_supply_temp = data_dict[f'{temp_type_prefix}_main_supply_온도_c']
            data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c'] = np.where(is_active_roll, base_supply_temp + np.random.normal(0,0.1,num_rows), base_supply_temp + np.random.uniform(0.5,2,num_rows) )
            data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'] = data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c'] + \
                                                                np.where(is_active_roll, (data_dict[load_key]/(active_count_series+1e-6))/(150000+1e-6) * 5 + np.random.normal(0,0.2,num_rows) , np.random.uniform(0.1,0.5,num_rows))
            data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c'] = outdoor_temps + 5 + np.random.normal(0,0.5,num_rows)
            data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'] = data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c'] + \
                                                                    np.where(is_active_roll, 3 + np.random.normal(0,0.3,num_rows), np.random.uniform(0.1,0.5,num_rows))

            data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'], data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+0.2, data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+12)
            data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'], data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+0.2, data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+10)

    for k in range(num_ct):
        data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] = outdoor_temps + 3 + np.random.normal(0,0.5,num_rows)
        data_dict[f'cooling_tower_{k}_return_냉각수온도_c'] = data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + np.random.uniform(3,7,num_rows)
        data_dict[f'cooling_tower_{k}_supply_수조레벨_percent'] = np.random.uniform(70,95,num_rows)
        data_dict[f'cooling_tower_{k}_return_냉각수온도_c'] = np.clip(data_dict[f'cooling_tower_{k}_return_냉각수온도_c'], data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + 1.0, data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + 15)

    df = pd.DataFrame(data_dict)
    print(f"상세 더미 데이터 생성 완료. DataFrame Shape: {df.shape}")
    return df


class OACModel:
    def __init__(self, num_oac_units_in_system=14):
        self.num_oac_units_in_system = num_oac_units_in_system # 시스템 내 총 OAC 대수
        self.enthalpy_calculator = EnthalpyCalculator()
        self.coil_order = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']

        self.trained_models = {} # {oac_unit_id: {coil_type: model}}
        self.feature_names_per_coil = {} # {oac_unit_id: {coil_type: [feature_names]}}
        self.design_delta_h = {
            'preheating': 15000, 'precooling': -20000, 'cooling': -25000,
            'reheating': 12000, 'humidification': 8000
        }

    def train_coil_models(self, df_train_data_full):
        print("\n--- OAC 코일 모델 학습 시작 (OAC 유닛별 독립 모델) ---")

        df_train_data_full['outdoor_enthalpy'] = self.enthalpy_calculator.calculate_enthalpy(
            df_train_data_full['outdoor_temp_c'], df_train_data_full['outdoor_rh_percent']
        )

        for oac_idx in range(self.num_oac_units_in_system):
            self.trained_models[oac_idx] = {}
            self.feature_names_per_coil[oac_idx] = {}
            print(f"  OAC 유닛 {oac_idx} 모델 학습 중...")

            for coil_type in self.coil_order:
                target_col = f'oac_{oac_idx}_{coil_type}_coil_open_rate'
                if target_col not in df_train_data_full.columns:
                    print(f"    경고: OAC {oac_idx}의 타겟 컬럼 {target_col}이 없어 {coil_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                    continue

                temp_df_train = df_train_data_full.copy()
                current_features_generic_names = []
                current_features_specific_names = []

                current_features_generic_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])
                current_features_specific_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])

                # 이전 코일 출구 상태 및 현재 코일 설정값 추가
                if coil_type == 'preheating': # 첫 코일
                    current_features_generic_names.append('current_coil_set_point_temp_c')
                    current_features_specific_names.append(f'oac_{oac_idx}_{coil_type}_set_point_temp_c')
                else: # preheating이 아닌 코일 (이전 코일 존재)
                    prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                    prev_outlet_temp_col_specific = f'oac_{oac_idx}_{prev_coil}_후단온도_c'
                    prev_outlet_rh_col_specific = f'oac_{oac_idx}_{prev_coil}_후단습도_percent'
                    prev_outlet_h_col_calculated_specific = f'oac_{oac_idx}_{prev_coil}_후단엔탈피_calculated'

                    temp_df_train[prev_outlet_h_col_calculated_specific] = self.enthalpy_calculator.calculate_enthalpy(
                        temp_df_train[prev_outlet_temp_col_specific], temp_df_train[prev_outlet_rh_col_specific]
                    )
                    current_features_generic_names.extend([f'prev_coil_outlet_temp_c', f'prev_coil_outlet_rh_percent', f'prev_coil_outlet_h_actual'])
                    current_features_specific_names.extend([prev_outlet_temp_col_specific, prev_outlet_rh_col_specific, prev_outlet_h_col_calculated_specific])

                    current_features_generic_names.append('current_coil_set_point_temp_c') # 모든 코일 목표는 온도로 통일
                    current_features_specific_names.append(f'oac_{oac_idx}_{coil_type}_set_point_temp_c')

                self.feature_names_per_coil[oac_idx][coil_type] = current_features_generic_names

                X = temp_df_train[current_features_specific_names].fillna(method='ffill').fillna(method='bfill')
                X.columns = current_features_generic_names # 컬럼명을 일반화된 이름으로 변경
                y = temp_df_train[target_col]

                if X.empty or y.empty or X.shape[0] != y.shape[0]:
                    print(f"    오류: OAC {oac_idx} - {coil_type} 특징/타겟 데이터 문제로 학습 건너<0xEB><0><0x84>니다.")
                    continue

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                model = RandomForestRegressor(n_estimators=20, random_state=42, n_jobs=-1, max_depth=8, min_samples_split=10, min_samples_leaf=5) # 모델 간소화
                model.fit(X_train, y_train)

                # y_pred_test = model.predict(X_test)
                # mse_test = mean_squared_error(y_test, y_pred_test)
                # print(f"    OAC {oac_idx} - {coil_type} 모델 학습 완료. Test MSE: {mse_test:.4f}")
                self.trained_models[oac_idx][coil_type] = model
        print("--- OAC 코일 모델 학습 완료 ---")

    def predict_coil_open_rates(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_all_coils, modified_oac_coil_states=None):
        """14개 OAC 유닛 전체에 대한 코일 개도율 및 상태를 예측합니다."""
        if not self.trained_models or not self.feature_names_per_coil:
            # print("경고: OAC 모델 또는 피처 정보가 학습되지 않았습니다. 모든 OAC에 규칙 기반 예측 적용.")
            all_oac_predictions = {}
            for oac_idx in range(self.num_oac_units_in_system):
                # modified_states_this_oac = modified_oac_coil_states.get(oac_idx, {}) if modified_oac_coil_states else {} # OAC별 수정 허용 시
                all_oac_predictions[oac_idx] = self._predict_single_oac_rule_based(
                    outdoor_temp_c, outdoor_rh_percent, oac_set_points_all_coils,
                    modified_oac_coil_states.get(f'oac_{oac_idx}') if modified_oac_coil_states else None # OAC별 수정값 전달
                )
            return all_oac_predictions

        if modified_oac_coil_states is None: modified_oac_coil_states = {}

        all_oac_results = {}
        outdoor_enthalpy_val = self.enthalpy_calculator.calculate_enthalpy(outdoor_temp_c, outdoor_rh_percent)

        for oac_idx in range(self.num_oac_units_in_system):
            current_oac_unit_results = {}
            current_temp_c_for_sim = outdoor_temp_c
            current_rh_percent_for_sim = outdoor_rh_percent

            prev_coil_actual_outlet_temp_for_pred = outdoor_temp_c
            prev_coil_actual_outlet_rh_for_pred = outdoor_rh_percent

            oac_specific_modified_states = modified_oac_coil_states.get(f'oac_{oac_idx}', {}) # 해당 OAC에 대한 수정사항

            for coil_type in self.coil_order:
                inlet_temp_c_sim = current_temp_c_for_sim
                inlet_rh_percent_sim = current_rh_percent_for_sim
                inlet_h_j_kg_sim = self.enthalpy_calculator.calculate_enthalpy(inlet_temp_c_sim, inlet_rh_percent_sim)
                predicted_open_rate = 0.0

                if coil_type in oac_specific_modified_states:
                    predicted_open_rate = oac_specific_modified_states[coil_type]
                elif oac_idx in self.trained_models and coil_type in self.trained_models[oac_idx]:
                    model = self.trained_models[oac_idx][coil_type]
                    generic_feature_names = self.feature_names_per_coil.get(oac_idx, {}).get(coil_type)
                    if not generic_feature_names:
                        predicted_open_rate = 0.0
                    else:
                        feature_values_map = {
                            'outdoor_temp_c': outdoor_temp_c, 'outdoor_rh_percent': outdoor_rh_percent, 'outdoor_enthalpy': outdoor_enthalpy_val,
                            'prev_coil_outlet_temp_c': prev_coil_actual_outlet_temp_for_pred,
                            'prev_coil_outlet_rh_percent': prev_coil_actual_outlet_rh_for_pred,
                            'prev_coil_outlet_h_actual': self.enthalpy_calculator.calculate_enthalpy(prev_coil_actual_outlet_temp_for_pred, prev_coil_actual_outlet_rh_for_pred),
                            'current_coil_set_point_temp_c': oac_set_points_all_coils.get(coil_type, inlet_temp_c_sim), # 모든 코일 목표는 온도
                        }
                        feature_values_for_pred_list = [feature_values_map[name] for name in generic_feature_names]
                        try:
                            input_df = pd.DataFrame([feature_values_for_pred_list], columns=generic_feature_names)
                            predicted_open_rate = model.predict(input_df)[0]
                            predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)
                        except Exception: predicted_open_rate = 0.0
                else: predicted_open_rate = 0.0

                actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
                outlet_h_j_kg_sim = inlet_h_j_kg_sim + actual_delta_h_j_kg
                current_temp_c_for_sim, current_rh_percent_for_sim = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg_sim, inlet_temp_c_sim)

                current_oac_unit_results[coil_type] = {
                    'open_rate': predicted_open_rate,
                    'inlet_temp_c': inlet_temp_c_sim, 'inlet_rh_percent': inlet_rh_percent_sim, 'inlet_h_j_kg': inlet_h_j_kg_sim,
                    'outlet_temp_c': current_temp_c_for_sim, 'outlet_rh_percent': current_rh_percent_for_sim, 'outlet_h_j_kg': outlet_h_j_kg_sim,
                    'delta_h_j_kg': actual_delta_h_j_kg,
                    'target_set_point_temp_c': oac_set_points_all_coils.get(coil_type, 'N/A')
                }
                prev_coil_actual_outlet_temp_for_pred = current_temp_c_for_sim
                prev_coil_actual_outlet_rh_for_pred = current_rh_percent_for_sim

            current_oac_unit_results['final_outlet_temp_c'] = current_temp_c_for_sim
            current_oac_unit_results['final_outlet_rh_percent'] = current_rh_percent_for_sim
            current_oac_unit_results['final_outlet_h_j_kg'] = self.enthalpy_calculator.calculate_enthalpy(current_temp_c_for_sim, current_rh_percent_for_sim)
            all_oac_results[f'oac_{oac_idx}'] = current_oac_unit_results
        return all_oac_results

    def _predict_single_oac_rule_based(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_all_coils, modified_coil_states_this_oac=None):
        """단일 OAC 유닛에 대한 규칙 기반 예측 (내부 헬퍼)"""
        if modified_coil_states_this_oac is None: modified_coil_states_this_oac = {}

        current_temp_c = outdoor_temp_c
        current_rh_percent = outdoor_rh_percent
        current_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(current_temp_c, current_rh_percent)
        unit_results = {}

        for coil_type in self.coil_order:
            inlet_temp_c, inlet_rh_percent, inlet_h_j_kg = current_temp_c, current_rh_percent, current_h_j_kg
            target_set_point_temp_c = oac_set_points_all_coils.get(coil_type) # 모든 코일 목표는 온도
            predicted_open_rate, actual_delta_h_j_kg = 0.0, 0.0

            if coil_type in modified_coil_states_this_oac:
                predicted_open_rate = modified_coil_states_this_oac[coil_type]
            elif target_set_point_temp_c is not None:
                # 목표 온도 도달을 위한 필요 엔탈피 변화량 계산
                target_rh_for_sp = inlet_rh_percent # 단순화: 목표 온도 도달 시 습도 현재와 동일 가정
                if coil_type == 'humidification' and target_set_point_temp_c > inlet_temp_c:
                     target_rh_for_sp = min(inlet_rh_percent + (target_set_point_temp_c - inlet_temp_c) * 5, 95)

                target_outlet_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(target_set_point_temp_c, target_rh_for_sp)
                required_delta_h_j_kg = target_outlet_h_j_kg - inlet_h_j_kg

                if self.design_delta_h[coil_type] != 0:
                    if (self.design_delta_h[coil_type] > 0 and required_delta_h_j_kg > 0) or \
                       (self.design_delta_h[coil_type] < 0 and required_delta_h_j_kg < 0):
                        predicted_open_rate = abs(required_delta_h_j_kg / self.design_delta_h[coil_type])
                predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)

            actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            outlet_h_j_kg = inlet_h_j_kg + actual_delta_h_j_kg
            current_temp_c, current_rh_percent = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg, inlet_temp_c)
            current_h_j_kg = outlet_h_j_kg
            unit_results[coil_type] = {
                'open_rate': predicted_open_rate, 'inlet_temp_c': inlet_temp_c, 'inlet_rh_percent': inlet_rh_percent,
                'inlet_h_j_kg': inlet_h_j_kg, 'outlet_temp_c': current_temp_c, 'outlet_rh_percent': current_rh_percent,
                'outlet_h_j_kg': current_h_j_kg, 'delta_h_j_kg': actual_delta_h_j_kg,
                'target_set_point_temp_c': target_set_point_temp_c
            }
        unit_results['final_outlet_temp_c'], unit_results['final_outlet_rh_percent'], unit_results['final_outlet_h_j_kg'] = current_temp_c, current_rh_percent, current_h_j_kg
        return unit_results


class ChillerModel: # 이전과 동일하게 유지 (시스템 레벨 예측)
    def __init__(self):
        self.trained_models = {}
        self.feature_names_chiller = {}

    def train_pressure_valve_models(self, df_train_data):
        print("\n--- 냉동기 차압/밸브 모델 학습 시작 ---")
        for chiller_type in ["low_temp", "high_temp"]:
            print(f"  {chiller_type}_chiller 시스템 모델 학습 중...")
            load_col_for_training = 'total_cooling_load_watts' if chiller_type == "low_temp" else 'total_precooling_load_watts'
            active_count_col_for_training = f'{chiller_type}_chiller_active_count'
            current_feature_cols = [load_col_for_training, active_count_col_for_training]
            self.feature_names_chiller[chiller_type] = current_feature_cols
            target_cols_for_training = [f'{chiller_type}_chiller_main_차압압력_bar',
                                        f'{chiller_type}_chiller_main_차압개도율_percent',
                                        f'{chiller_type}_chiller_sub_차압개도율_percent']
            if not all(col in df_train_data.columns for col in current_feature_cols + target_cols_for_training):
                print(f"    경고: {chiller_type} 학습에 필요한 컬럼 부족. 건너<0xEB><0><0x84>니다.")
                continue
            X = df_train_data[current_feature_cols].fillna(method='ffill').fillna(method='bfill')
            type_models = {}
            for target_col in target_cols_for_training:
                y = df_train_data[target_col]
                if X.empty or y.empty or X.shape[0] != y.shape[0]: continue
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                model = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1, max_depth=8, min_samples_split=8,min_samples_leaf=4)
                model.fit(X_train, y_train)
                output_key = target_col.replace(f'{chiller_type}_chiller_', '')
                # print(f"    {output_key} 모델 학습 완료. MSE 생략")
                type_models[output_key] = model
            self.trained_models[chiller_type] = type_models
        print("--- 냉동기 차압/밸브 모델 학습 완료 ---")

    def predict_pressure_valve(self, current_total_load_watts, num_active_chillers_of_type, chiller_type="low_temp"):
        if chiller_type not in self.trained_models or not self.trained_models[chiller_type] or \
           chiller_type not in self.feature_names_chiller:
            return {"main_차압압력_bar": 1.5, "main_차압개도율_percent": 60.0, "sub_차압개도율_percent": 50.0}
        models_for_type = self.trained_models[chiller_type]
        feature_cols_for_prediction = self.feature_names_chiller[chiller_type]
        input_df_for_pred = pd.DataFrame([[current_total_load_watts, num_active_chillers_of_type]], columns=feature_cols_for_prediction)
        predictions = {}
        for output_key, model in models_for_type.items():
            try: predictions[output_key] = model.predict(input_df_for_pred)[0]
            except Exception:
                if "차압압력" in output_key: predictions[output_key] = 1.5
                elif "차압개도율" in output_key: predictions[output_key] = 60.0
                else: predictions[output_key] = 0
        expected_output_keys = ["main_차압압력_bar", "main_차압개도율_percent", "sub_차압개도율_percent"]
        for key in expected_output_keys:
            if key not in predictions:
                if "차압압력" in key: predictions[key] = 1.5
                else: predictions[key] = 60.0
        return predictions


class ChillerOptimizer: # 이전과 동일
    def __init__(self, num_low_temp_chillers=8, num_high_temp_chillers=8):
        self.num_low_temp_chillers = num_low_temp_chillers
        self.num_high_temp_chillers = num_high_temp_chillers
        self.chiller_specs = {
            "low_temp": [{"id": i, "max_capacity_watts": 200000, "cop_points": {0.3: 3.0, 0.55: 3.8, 0.8: 3.5, 1.0: 3.0}} for i in range(num_low_temp_chillers)],
            "high_temp": [{"id": i, "max_capacity_watts": 250000, "cop_points": {0.3: 3.2, 0.55: 4.0, 0.8: 3.7, 1.0: 3.2}} for i in range(num_high_temp_chillers)]
        }
        for type_key in self.chiller_specs:
            self.chiller_specs[type_key] = sorted(self.chiller_specs[type_key], key=lambda x: self._get_cop_at_load(x['cop_points'], 0.55), reverse=True)
        self.target_load_percentage = 0.55
        self.min_load_percentage = 0.30
        self.max_load_percentage = 0.90

    def _get_cop_at_load(self, cop_points, load_percentage):
        if not cop_points: return 3.0
        load_percentage = np.clip(load_percentage, 0.0, 1.0)
        if load_percentage == 0: return 0
        sorted_load_points = sorted(cop_points.keys())
        if load_percentage <= sorted_load_points[0]: return cop_points[sorted_load_points[0]]
        if load_percentage >= sorted_load_points[-1]: return cop_points[sorted_load_points[-1]]
        for i in range(len(sorted_load_points) - 1):
            x1, y1 = sorted_load_points[i], cop_points[sorted_load_points[i]]
            x2, y2 = sorted_load_points[i+1], cop_points[sorted_load_points[i+1]]
            if x1 <= load_percentage < x2:
                return y1 + (y2 - y1) * (load_percentage - x1) / (x2 - x1) if (x2-x1) != 0 else y1
        return cop_points[sorted_load_points[-1]]

    def optimize_chiller_operation(self, required_total_load_watts, chiller_type="low_temp"):
        specs_list = self.chiller_specs[chiller_type]
        num_total_chillers = len(specs_list)
        best_config = {'num_active': 0, 'active_chiller_ids': [], 'load_percentages': [], 'total_power': float('inf'), 'avg_load_percentage': 0, 'meets_target_load_rate': False}

        if required_total_load_watts <= 0: return 0, [], [], 0, 0, True

        for num_active in range(1, num_total_chillers + 1):
            current_chiller_selection_specs = specs_list[:num_active]
            total_max_cap_selected = sum(s['max_capacity_watts'] for s in current_chiller_selection_specs)
            if total_max_cap_selected * self.max_load_percentage < required_total_load_watts: continue
            if total_max_cap_selected * self.min_load_percentage > required_total_load_watts and required_total_load_watts > 0:
                if num_active > 1 : continue

            load_per_chiller_target = required_total_load_watts / num_active
            current_load_percentages, current_total_power, possible_combination, active_ids_temp = [], 0, True, []
            for chiller_spec in current_chiller_selection_specs:
                active_ids_temp.append(chiller_spec['id'])
                load_percentage_this_chiller = load_per_chiller_target / chiller_spec['max_capacity_watts']
                if not (self.min_load_percentage <= load_percentage_this_chiller <= self.max_load_percentage):
                    possible_combination = False; break
                current_load_percentages.append(load_percentage_this_chiller)
                cop = self._get_cop_at_load(chiller_spec['cop_points'], load_percentage_this_chiller)
                if cop <= 0: possible_combination = False; break
                current_total_power += (chiller_spec['max_capacity_watts'] * load_percentage_this_chiller) / cop

            if not possible_combination: continue
            avg_load_percentage = sum(current_load_percentages) / num_active if num_active > 0 else 0
            meets_target = abs(avg_load_percentage - self.target_load_percentage) < 0.05

            is_better = False
            if not best_config['active_chiller_ids']: is_better = True
            elif best_config['meets_target_load_rate'] and meets_target:
                if current_total_power < best_config['total_power']: is_better = True
            elif meets_target and not best_config['meets_target_load_rate']: is_better = True
            elif not best_config['meets_target_load_rate'] and not meets_target:
                if abs(avg_load_percentage - self.target_load_percentage) < abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    is_better = True
                elif abs(avg_load_percentage - self.target_load_percentage) == abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    if current_total_power < best_config['total_power']: is_better = True

            if is_better:
                best_config.update({'num_active': num_active, 'active_chiller_ids': sorted(active_ids_temp), 'load_percentages': current_load_percentages,
                                    'total_power': current_total_power, 'avg_load_percentage': avg_load_percentage, 'meets_target_load_rate': meets_target})

        if not best_config['active_chiller_ids'] and required_total_load_watts > 0 :
            num_active = num_total_chillers
            active_ids_temp = [s['id'] for s in specs_list]
            load_per_chiller = required_total_load_watts / num_active if num_active > 0 else 0
            current_load_percentages, current_total_power = [], 0
            for chiller_spec in specs_list:
                load_percentage = load_per_chiller / chiller_spec['max_capacity_watts'] if chiller_spec['max_capacity_watts'] > 0 else 0
                effective_load_perc_for_cop = np.clip(load_percentage, self.min_load_percentage, self.max_load_percentage)
                current_load_percentages.append(load_percentage)
                cop = self._get_cop_at_load(chiller_spec['cop_points'], effective_load_perc_for_cop)
                if cop <= 0: cop = 1.0
                current_total_power += (chiller_spec['max_capacity_watts'] * load_percentage) / cop

            avg_lp_fallback = sum(lp for lp in current_load_percentages if lp <=1.0) / num_active if num_active > 0 else 0
            best_config.update({'num_active': num_active, 'active_chiller_ids': sorted(active_ids_temp), 'load_percentages': current_load_percentages,
                                'total_power': current_total_power, 'avg_load_percentage': avg_lp_fallback,
                                'meets_target_load_rate': abs(avg_lp_fallback - self.target_load_percentage) < 0.10})
        return best_config['num_active'], best_config['active_chiller_ids'], best_config['load_percentages'], best_config['total_power'], best_config['avg_load_percentage'], best_config['meets_target_load_rate']

class CoolingTowerOptimizer: # 이전과 동일
    def __init__(self, num_towers=9):
        self.tower_specs = [{"id": i, "max_capacity_watts": 300000 + np.random.randint(-2,3)*10000, "fan_power_watts": 7000 + np.random.randint(-5,6)*100} for i in range(num_towers)]
        self.num_towers = len(self.tower_specs)
        self.sorted_towers = sorted(self.tower_specs, key=lambda x: x['fan_power_watts'] / x['max_capacity_watts'] if x['max_capacity_watts'] > 0 else float('inf'))

    def optimize_tower_operation(self, total_heat_rejection_load_watts):
        active_tower_ids, current_total_capacity, current_total_fan_power = [], 0, 0
        if total_heat_rejection_load_watts <= 0: return [], 0, 0, 0
        for tower in self.sorted_towers:
            if current_total_capacity >= total_heat_rejection_load_watts: break
            active_tower_ids.append(tower['id']); current_total_capacity += tower['max_capacity_watts']; current_total_fan_power += tower['fan_power_watts']
        return sorted(active_tower_ids), current_total_fan_power, current_total_capacity, len(active_tower_ids)

class HVACSystemFacade:
    def __init__(self, oac_model_instance, chiller_model_instance):
        self.oac_model = oac_model_instance # OACModel 인스턴스 (내부에 14개 OAC 모델 포함)
        self.chiller_model = chiller_model_instance
        self.chiller_optimizer = ChillerOptimizer()
        self.cooling_tower_optimizer = CoolingTowerOptimizer()
        self.air_mass_flow_rate_per_oac_unit_kg_s = 12.0
        # self.num_oac_units = 14 # OACModel 내부에 num_oac_units_in_system으로 관리

    def predict_and_optimize(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_all_coils,
                             chilled_water_supply_set_temp, modified_oac_coil_states_all_units=None):
        # 1. OAC 코일 개도율 및 각 단 출구 상태 예측 (14개 OAC 전체)
        # modified_oac_coil_states_all_units는 dict 형태: {'oac_0': {'cooling':0.8}, 'oac_1':{...}}
        oac_predictions_all_units = self.oac_model.predict_coil_open_rates(
            outdoor_temp_c, outdoor_rh_percent, oac_set_points_all_coils, modified_oac_coil_states_all_units
        )

        # 2. 냉동기 부하 계산 (14개 OAC 부하 합산)
        total_precooling_load_watts_calculated = 0
        total_cooling_load_watts_calculated = 0
        for oac_id_key, unit_prediction in oac_predictions_all_units.items():
            total_precooling_load_watts_calculated += abs(unit_prediction.get('precooling', {}).get('delta_h_j_kg', 0) * self.air_mass_flow_rate_per_oac_unit_kg_s)
            total_cooling_load_watts_calculated += abs(unit_prediction.get('cooling', {}).get('delta_h_j_kg', 0) * self.air_mass_flow_rate_per_oac_unit_kg_s)

        # 3. 저온 냉동기 최적화 및 예측
        optim_low_ch_count, active_low_ids, load_dist_low, power_low, avg_load_low, meets_target_low = \
            self.chiller_optimizer.optimize_chiller_operation(total_cooling_load_watts_calculated, chiller_type="low_temp")
        chiller_preds_low_temp = {}
        if optim_low_ch_count > 0 :
            chiller_preds_low_temp = self.chiller_model.predict_pressure_valve(
                total_cooling_load_watts_calculated, optim_low_ch_count, chiller_type="low_temp"
            )

        # 4. 고온 냉동기 최적화 및 예측
        optim_high_ch_count, active_high_ids, load_dist_high, power_high, avg_load_high, meets_target_high = \
            self.chiller_optimizer.optimize_chiller_operation(total_precooling_load_watts_calculated, chiller_type="high_temp")
        chiller_preds_high_temp = {}
        if optim_high_ch_count > 0 :
            chiller_preds_high_temp = self.chiller_model.predict_pressure_valve(
                total_precooling_load_watts_calculated, optim_high_ch_count, chiller_type="high_temp"
            )

        total_heat_rejection_watts = total_precooling_load_watts_calculated + total_cooling_load_watts_calculated + power_low + power_high
        active_tower_ids, tower_fan_power, tower_total_capacity, active_tower_count = \
            self.cooling_tower_optimizer.optimize_tower_operation(total_heat_rejection_watts)

        return {
            "oac_predictions_all_units": oac_predictions_all_units, # 14개 OAC 전체 예측 결과
            "calculated_loads": {"total_precooling_load_watts": total_precooling_load_watts_calculated,
                                 "total_cooling_load_watts": total_cooling_load_watts_calculated},
            "low_temp_chiller_optimization": {"optimal_active_count": optim_low_ch_count, "active_chiller_ids": active_low_ids, "load_distribution_percentage_per_active_chiller": load_dist_low, "total_estimated_power_watts": power_low, "average_load_percentage_of_active_chillers": avg_load_low, "meets_target_load_rate": meets_target_low, "predictions": chiller_preds_low_temp},
            "high_temp_chiller_optimization": {"optimal_active_count": optim_high_ch_count, "active_chiller_ids": active_high_ids, "load_distribution_percentage_per_active_chiller": load_dist_high, "total_estimated_power_watts": power_high, "average_load_percentage_of_active_chillers": avg_load_high, "meets_target_load_rate": meets_target_high, "predictions": chiller_preds_high_temp},
            "cooling_tower_optimization": {"total_heat_rejection_load_watts": total_heat_rejection_watts, "active_tower_ids": active_tower_ids, "active_tower_count": active_tower_count, "total_fan_power_watts": tower_fan_power, "total_active_capacity_watts": tower_total_capacity}
        }

if __name__ == '__main__':
    print("--- 1. 상세 더미 데이터 생성 중 ---")
    # 행 수를 줄여 테스트 시간 단축 (예: 200~500)
    # OAC 모델 학습 시 많은 모델(14*5=70)을 학습하므로 시간 소요.
    dummy_df = generate_dummy_data(num_rows=200, num_oac=14, num_low_ch=8, num_high_ch=8, num_ct=9)

    print("\n--- 2. 모델 객체 생성 및 학습 ---")
    oac_model_inst = OACModel(num_oac_units_in_system=14) # 시스템 내 OAC 대수 전달
    oac_model_inst.train_coil_models(dummy_df.copy())

    chiller_model_inst = ChillerModel()
    chiller_model_inst.train_pressure_valve_models(dummy_df.copy())

    print("\n--- 3. HVAC 시스템 Facade 생성 (학습된 모델 사용) ---")
    hvac_system = HVACSystemFacade(oac_model_instance=oac_model_inst,
                                   chiller_model_instance=chiller_model_inst)
    print("Facade 객체 생성 완료.")

    print("\n" + "="*20 + " 학습된 모델 기반 하절기 시나리오 실행 " + "="*20)
    outdoor_temp_summer = 32.0
    outdoor_humidity_summer = 75.0
    # 모든 OAC 코일의 목표는 온도로 설정
    oac_target_set_points_summer_all_coils = {
        'preheating': 18.0,
        'precooling': 20.0,
        'cooling': 12.0,
        'reheating': 23.0,
        'humidification': 23.5 # 가습 코일 목표 '온도' (예: 재열 후 약간 더 높게)
    }
    chilled_water_supply_temp_set_summer = 7.0

    print(f"\n[입력 조건]")
    print(f"  외기 온도: {outdoor_temp_summer}°C, 외기 상대 습도: {outdoor_humidity_summer}%")
    print(f"  OAC 코일별 목표 설정 (모든 OAC 공통):")
    for coil, sp_val in oac_target_set_points_summer_all_coils.items():
        print(f"    - {coil.capitalize()} 목표 온도: {sp_val}°C")
    print("-" * 50)

    summer_results = hvac_system.predict_and_optimize(
        outdoor_temp_summer, outdoor_humidity_summer,
        oac_target_set_points_summer_all_coils,
        chilled_water_supply_temp_set_summer
    )

    print("\n[OAC 예측 결과 요약 (14개 OAC 전체)]")
    oac_all_preds = summer_results["oac_predictions_all_units"]
    if oac_all_preds:
        # 첫번째 OAC (oac_0) 상세 결과 (예시)
        first_oac_key = list(oac_all_preds.keys())[0]
        print(f"  대표 OAC ({first_oac_key}) 코일별 예측:")
        for coil_name in oac_model_inst.coil_order:
            data = oac_all_preds[first_oac_key].get(coil_name)
            if isinstance(data, dict):
                target_display = data.get('target_set_point_temp_c', 'N/A')
                print(f"    코일: {coil_name.capitalize()} (목표: {target_display}°C)")
                print(f"      예측 개도율: {data.get('open_rate', 0.0):.3f} ({(data.get('open_rate', 0.0)*100):.1f}%)")
        print(f"  대표 OAC ({first_oac_key}) 최종 토출: {oac_all_preds[first_oac_key].get('final_outlet_temp_c'):.1f}°C, {oac_all_preds[first_oac_key].get('final_outlet_rh_percent'):.1f}%")

        # 전체 OAC 평균 최종 토출 온도 (예시)
        avg_final_temp = np.mean([val['final_outlet_temp_c'] for val in oac_all_preds.values() if 'final_outlet_temp_c' in val])
        print(f"  전체 OAC 평균 최종 토출 온도 (근사): {avg_final_temp:.1f}°C")


    print("\n[계산된 총 시스템 부하 (OAC 14대 기준)]")
    loads = summer_results["calculated_loads"]
    print(f"  총 예냉 코일 부하 (고온 냉동기 담당): {loads['total_precooling_load_watts']:,.0f} Watts")
    print(f"  총 주 냉각 코일 부하 (저온 냉동기 담당): {loads['total_cooling_load_watts']:,.0f} Watts")

    print("\n[저온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    low_temp_ch_opt = summer_results["low_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {low_temp_ch_opt['optimal_active_count']} 대 (IDs: {low_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {low_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예측된 시스템 차압/밸브: {low_temp_ch_opt['predictions']}")

    print("\n[고온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    high_temp_ch_opt = summer_results["high_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {high_temp_ch_opt['optimal_active_count']} 대 (IDs: {high_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {high_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예측된 시스템 차압/밸브: {high_temp_ch_opt['predictions']}")

    print("\n[냉각탑 최적화 결과]")
    ct_opt = summer_results["cooling_tower_optimization"]
    print(f"  총 방열 부하: {ct_opt['total_heat_rejection_load_watts']:,.0f} Watts")
    print(f"  최적 가동 대수: {ct_opt['active_tower_count']} 대 (IDs: {ct_opt['active_tower_ids']})")
    print("="*50)